In [3]:
import pandas as pd
from germansentiment import SentimentModel
import torch

/Users/marc/.pyenv/versions/3.9.1/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
dataset = pd.read_csv("../data/raw/songs_complete_final.csv")

In [5]:
model = SentimentModel()

In [6]:
mps_device = torch.device("mps")
model.model.to(mps_device)
model.device = "mps"

In [7]:
classes, probabilities = model.predict_sentiment(dataset["lyrics"][:10], output_probabilities = True)

In [8]:
from pprint import pprint
pprint(list(zip(classes, probabilities, dataset["lyrics"][:10])))

[('negative',
  [['positive', 0.012294584885239601],
   ['negative', 0.9867066740989685],
   ['neutral', 0.000998738338239491]],
  'Powpow  Dicka, das kein Rap mehr, das ist Kindergarten Ihr wollt euch '
  'treffen, doch dann lass ich eure Mütter warten Es ist KUKU TEAM, bringe '
  'euch gemischte Karten Ihr könnt nicht rappen, dafür könnt ihr\u205fso\u205f'
  'wie\u205fBitches blasen Ich geb\u205fdem Bastard jetzt\u205feinn '
  'Gnadenschuss Ihr seid keine Straßenjungs, ihr färbt euch die Haare bunt '
  'Hör, mein Magen knurrt, Köfte im Fladenbrot Mach ein Nickerchen auf Sofa so '
  'wie Baba Packs, Waage, Flex, Dicka, dis kein StanniKurs Was ist das für '
  'Blick Moruk, als ob deine Mami furzt Heiße Aura wie Solarium Besser '
  'verschwinde aus mein Radius Immer gute Lage so wie Mehringdamm Bares über '
  'Telegram, Ware wie ein Chemiker Ihr Huren seid mir scheißegal Ob legal, '
  'illegal, ist mir scheißegal'),
 ('negative',
  [['positive', 0.012251140549778938],
   ['negative', 0.

### Fine Tune Classifier

In [9]:
import pandas as pd
ds = pd.read_csv("/Users/marc/Desktop/statistik-1-team-2/data/raw/songs_complete_final.csv")
ds.dropna(subset=["genre_cat"], inplace=True)
ds["genre_cat"].unique()
factorized = pd.factorize(ds["genre_cat"])
print(factorized)
ds["labels"] = factorized[0]
ds.to_csv("/Users/marc/Desktop/statistik-1-team-2/data/raw/songs_complete_final_no_nan.csv", index=False)

(array([0, 0, 0, ..., 1, 1, 1]), Index(['Rock', 'Rap', 'Pop', 'Schlager'], dtype='object'))


In [10]:
from datasets import load_dataset
hf_dataset = load_dataset("csv", data_files="/Users/marc/Desktop/statistik-1-team-2/data/raw/songs_complete_final_no_nan.csv")
# hf_dataset=hf_dataset.rename_columns({"genre_cat": "labels"})
print(hf_dataset)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 244.98it/s]


Dataset csv downloaded and prepared to /Users/marc/.cache/huggingface/datasets/csv/default-06ad6ddce1ff2f54/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d. Subsequent calls will reuse this data.


100%|██████████| 1/1 [00:00<00:00, 430.67it/s]

DatasetDict({
    train: Dataset({
        features: ['artist', 'artist_id', 'album', 'album_id', 'release_date', 'title', 'full_title', 'song_id', 'lyrics', 'release_year', 'weekday', 'genre', 'genre_cat', 'word_count', 'labels'],
        num_rows: 16429
    })
})


In [11]:
split = hf_dataset["train"].train_test_split(test_size=0.2)
train_ds = split["train"]
test_ds = split["test"]

In [12]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
tokenizer = AutoTokenizer.from_pretrained("bert-base-german-cased")

In [13]:
def tokenize_function(examples):
    return tokenizer(examples["lyrics"], padding="max_length", truncation=True)

In [14]:
train_tokenized = train_ds.map(tokenize_function, batched=True)
test_tokenized = test_ds.map(tokenize_function, batched=True)

In [15]:
small_train_dataset = train_tokenized.shuffle(seed=42).select(range(1000))
small_test_dataset = test_tokenized.shuffle(seed=42).select(range(100))

In [16]:
id2label = {idx: val for idx, val in enumerate(factorized[1])}
print(id2label)
label2id = {val: key for (key, val) in id2label.items()}

{0: 'Rock', 1: 'Rap', 2: 'Pop', 3: 'Schlager'}


In [17]:
model = AutoModelForSequenceClassification.from_pretrained("bert-base-german-cased", num_labels=4, id2label=id2label, label2id=label2id)
# model.to("mps")

Some weights of the model checkpoint at bert-base-german-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-german-cased and a

In [29]:
!accelerate env

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)

Copy-and-paste the text below in your GitHub issue

- `Accelerate` version: 0.20.1
- Platform: macOS-13.4.1-arm64-arm-64bit
- Python version: 3.9.1
- Numpy version: 1.25.0
- PyTorch version (GPU?): 2.0.1 (False)
- PyTorch XPU available: False
- System RAM: 16.00 GB
- `Accelerate` default config:
	- compute_environment: LOCAL_MACHINE
	- distributed_type: NO
	- mixed_precision: fp16
	- use_cpu: True
	- num_processes: 1
	- machine_rank: 0
	- num_machines: 1
	- rdzv_backend: static
	- same_network: False
	- main_training_function: main
	- downcast_bf16: False
	- tpu_use_cluster: False
	- tpu_use_sudo: False


In [30]:
training_args = TrainingArguments(
    output_dir="bert_genre",
    use_mps_device=True,
    num_train_epochs=2,
    # learning_rate=2e-5,
    # per_device_train_batch_size=16,
    # per_device_eval_batch_size=16,
    # weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",)

ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.20.1`: Please run `pip install transformers[torch]` or `pip install accelerate -U`

In [ ]:
import numpy as np
import evaluate

accuraracy = evaluate.load("accuracy")
recall = evaluate.load("recall")
precision = evaluate.load("precision")
f1 = evaluate.load("f1")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    results = {}
    results.update(accuraracy.compute(predictions=predictions, references=labels))
    results.update(recall.compute(predictions=predictions, references=labels, average="micro"))
    results.update(precision.compute(predictions=predictions, references=labels, average="micro"))
    results.update(f1.compute(predictions=predictions, references=labels, average="micro"))
    return results

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_test_dataset,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

In [ ]:
from transformers import pipeline
pipe = pipeline(task="text-classification", model=model, tokenizer=tokenizer, return_all_scores=True, device="mps")



In [ ]:
toc1 = """
ch denke an das, was Du empfiehlst:
"Talent borrows, genius steals"
An das was Du mir anvertraust:
"Wähle dir deine fallen aus"
Ich glaub ich kanns erst jetzt verstehen
Wir müssen durch den Spiegel gehen
Danke auch für den Versuch
Das hier ist kein Wörterbuch
Du streichst mir über
Mein Gesicht
Gegen die Welt
Gegen den Strich
Meine Liebe
Dein Verzicht
Gegen die Welt
Gegen den Strich
Völker! Auf zum Gefecht!
Die Illusion wird Menschenrecht
Ich bin nicht allein in meiner Sucht
Vor den Spießern auf der Flucht
Du denkst an mich
Ich denk' an Dich
Gegen die Welt
Gegen den Strich
Unser Denken
Verbündet sich
Gegen die Welt
Gegen den Strich
Jeden Morgen, jede Nacht
Jeden Morgen, jede Nacht
Jeden Morgen, jede Nacht
Jeden Morgen, jede Nacht
Du streichst mir über
Mein Gesicht
Gegen die Welt
Gegen den Strich
Meine Liebe
Dein Verzicht
Gegen die Welt
Gegen den Strich
Du streichst mir über
Mein Gesicht
Gegen die Welt
Gegen den Strich
Meine Liebe
Dein Verzicht
Gegen die Welt
Gegen den Strich
"""

In [ ]:
kool1 = """Ich komm auf die Bühne leg Hand ans Mic
Fang langsam an und demolier dich am Ende wie Van Damme
Geh hol deine Gun dann, wärs fair
Doch bis dahin ist das einzge was du gewinnst nur Land man
Guck euer Gerede macht Müde wie der Sandmann
Ob Türke, Russe oder der Rest von Grand Cannes
Bei jedem finden diese Tracks Anklang, aber was ist das
Deine Bitch hat ein Loch wie'n Schuss von ner Pumpgun
Na dann viel Fun man, ich an deiner Stelle würd
Sie am randstehen lassen wie'n Wandschrank
Egal kein tam-tam, wahrt den Anstand meine
Texte ficken alles wie Matrosen auf Landgang
Für mich ist rappen wie wandern, s läuft
Und ich bediene gerne euer Team alleine die Bank lang
Danke an alle die mein Talent verkannt ham
Heute seht ihr wohl alles anders wie in nem Handstand, oder nicht?
Nie, niemals kriegt ihr mich weg, nie wi-ieder geh' ich
Keiner von euch kann's, wie di-ieser Typ ist da Vinci
Und das hier ist meine Mona-Lisa
Des Original, die Eins, vielmals von ihm kopiert
Aber niemals erreicht, ihr meint ihr seids
Doch kommt nicht ran an Mona-Lisa
Ich renn' durch Beats wie'n Athlet
Ich nehm' Wörter und Silben und kann sie im Takt dreh'n
Des ist'n Fetisch, ähnlich wie bei Leuten, die auf Leder und Lack stehen
Ich kann mich an meinen Texten nicht satt sehen
Ich hab' den übelsten dich-glatt-bügelnden Flow
Alles, was ich rappe, muss "Zack Zack Zack" gehen
Sie wollen mein Album mehr, als Miss Milian nackt sehen
Feature mit dir, ich muss ablehnen
Was geht, alle nenn'n dich Kackface
Als würde dein Kopf unmittelbar hinter dem Sack kleben
Mich will jeder wie 'ne Wiese plattmähen
Mich battlen lass', wie bei 'nem Liebesakt blähen
"""

In [ ]:
pipe(kool1)